# Getting started

The package we will use for our network analysis is `igraph`. There are also other packages out there for Python, such as `networkx` or `graphtool`, but will not use them during this course.

In [ ]:
import igraph as ig

Note that at any time, you can get some information by pressing Shift-Tab when your cursor is on some function.

We can create a graph ourselves, adding vertices and edges as we go.

In [ ]:
G = ig.Graph(directed=False)
G.add_vertices(n=4)
G.add_edges([(0, 1),
             (0, 2),
             (0, 3),
             (1, 2),
             (1, 3),
             (2, 3)])

We can get a summary of the graph, providing some basic information on the graph

In [ ]:
G.summary()

The result indicates this graph is undirected (indicated by the `U`) has 4 nodes and 6 edges. The information on the number of nodes and edges can also be obtained using functions.

In [ ]:
n = G.vcount()
m = G.ecount()
print('{0} nodes, {1} edges'.format(n, m))

In the remainder of this exercise, we will work with a built-in graph that is quite famous: the karate club network constructed by [Zachary (1977)](https://www.jstor.org/stable/3629752?seq=1#page_scan_tab_contents).

In [ ]:
G = ig.Graph.Famous('Zachary')

You can plot the graph easily

In [ ]:
G['layout'] = G.layout_fruchterman_reingold()
G.vs['color'] = 'gray'
G.es['color'] = 'gray'
ig.plot(G)

The most basic elements of any graph are the *nodes* and the connection between the nodes, called *edges*. Nodes are also called *vertices* and edges are also called *links*. These words will be used interchangebly. Vertices and edges are the terms that are most often used in graph theory, while nodes and links are more common in (social) network analysis. Sometimes, you also see the term *tie* or *arc* for referring to an edge.

In `igraph` the terms vertex and edges are used throughout, and they can be accessed through a so-called `VertexSequence` and `EdgeSequence`. You can make different selections of vertices and edges, either based on attributes or simply specifying specific (vertex or edge) indices. The functionality on vertex and edge sequences is quite extensive. The following just demonstrates one possibility.

In [ ]:
vs = G.vs[[0, 1, 2, 3]]
es = G.es.select(_within=vs)
vs['color'] = 'blue'
es['color'] = 'blue'
ig.plot(G)

The nodes that are linked to another node are called the *neighbors* of a node. The number of neighbours is called the *degree* of a node.

In [ ]:
neighbors = G.neighbors(4)
print(neighbors, G.degree(4))

# Paths

One of the most basic operations on any network is finding a *path* from one node to another node, not unlike finding a route from one city to another using some navigational software.

In [ ]:
edge_paths = G.get_shortest_paths(v=16, to=15, output='epath')
edge_paths

This retrieves a shortest path between node `16` and node `15`. It returns the *indices* of the edges because we set `output='epath'`. In this case there is only one path of 5 edges long. We can also get the endpoints of those edges, so that the path becomes a bit more clear.

In [ ]:
edge_path = G.es[edge_paths[0]];
[(edge.source, edge.target) for edge in edge_path]

We can also get the same path in terms of vertices

In [ ]:
vertex_path = G.vs[G.get_shortest_paths(v=16, to=15, output='vpath')[0]]
[v.index for v in vertex_path]

We can visualize this path as follows.

In [ ]:
G.vs['color'] = 'gray'
vertex_path['color'] = 'red'

G.es['color'] = 'gray'
G.es['width'] = 0.5
edge_path['color'] = 'red'
edge_path['width'] = 2

ig.plot(G)

Rather than determining the actual shortest paths, we may also simply be interested in the distance between two nodes.

In [ ]:
G.shortest_paths(source=16, target=15)

This only provides the actual length of the path (5 edges) rather than the actual path. We can also use this function to get the distance of all nodes to all other nodes. This is conveniently represented as a matrix, for which the `numpy` library is especially well suited.

In [ ]:
import numpy as np
distances = np.array(G.shortest_paths())
distances

The largest distance from a node to any other node is called the *eccentricity*. If a node has a very high eccentricity, it is a bit peripheral. If a node has a very low eccentricity, it means that it is relatively close to to all other nodes, and that it is in the center of the graph in a certain sense.

In [ ]:
eccentricity = distances.max(1)
ig.plot(G, vertex_size=5*(6 - eccentricity))

The minimum eccentricity is called the *radius* of the graph and the maximum eccentricity is called the *diameter*. The diameter of any graph is always larger than the radius and smaller than twice the radius.

The network we currently look at is connected, which is not necessarily the case. It can consist of multiple components. Let us delete a number of edges so that we get a network with two components.

In [ ]:
G.delete_edges(G.es.select(_between=[(4, 5, 6, 10), (0,)]))
ig.plot(G)

The path we constructed earlier is now no longer connected. In this case, there is no longer any path between the two nodes at all. The distance between the two nodes in hence infinitely large.

In [ ]:
G.shortest_paths(source=16, target=15)

There is now no longer a path between any two nodes, and the graph is then no longer said to be *connected*.

In [ ]:
G.is_connected()

The network is now said to be *disconnected* and the different parts of the network that still are connected are called *connected components*.

In [ ]:
components = G.clusters()
G.vs[components[0]]['color'] = 'red'
G.vs[components[1]]['color'] = 'blue'
ig.plot(G)

Usually, networks tend to have one large component, and many smaller components. In empirical networks, it is therefore common practice to restrict any further analyses to the largest connected component. Because the difference between the largest connected component and the other components is usually quite large, the largest connected component is sometimes also called the *giant component*.

In [ ]:
H = G.clusters().giant()
ig.plot(H, layout=H.layout_fruchterman_reingold())

# Cycles

If there are two paths going from one node to another node, we can join them together and make a *cycle*. We start again with a fresh graph.

In [ ]:
G = ig.Graph.Famous('Zachary')
G['layout'] = G.layout_fruchterman_reingold()

In [ ]:
cycle = G.es[G.get_eids(path=(24, 25, 23, 27, 24))]

G.es['color'] = 'gray'
G.es['width'] = 0.5
cycle['color'] = 'red'
cycle['width'] = 2

ig.plot(G)

This is a cycle of length 4, but longer cycles do exist in the graph. The smallest cycle of length 3 are of particular interest to the social sciences. A small group of three people is commonly called a *triad* in the social sciences. It is of particular interest because if two people have a friend in common, they are more likely to become friends themselves. This is known as *triadic closure*.

If there are many closed triads, i.e. many cycles of length 3, it suggests that many people have many friends in common. In particular, the network is then said to be *clustered*. The *clustering coefficient* for a node is defined as the proportion of neighbours that are connected amongst each other. A clustering coefficient of 1 then means that all the neighbours of a node are connected to each other, while a clustering coefficient of 0 means that no neigbhours are connected amongst each other. The overall clustering is then simply the average over the whole network.

In [ ]:
clustering = G.transitivity_local_undirected(mode="zero")

ig.plot(G, vertex_size=10*(np.array(clustering)+0.5))

As you can see, the nodes that are more clustered seem to be more peripheral. This is something that can be seen more
often. Nodes that have many neighbours usually have fewer connections between *all* their neighbours, so that they tend to have a lower clustering coefficient.

Nodes that have a higher clustering coefficient tend to be well embedded in the network. Nodes with a low clustering coefficient on the other hand tend to function as bridges, connecting different parts of the network. For example, when removing node 0 from the network, this disconnects the network.

In [ ]:
H = G.copy()
H.delete_vertices(0)
components = H.clusters()
ig.plot(components, layout=H.layout_fruchterman_reingold())

This means that node 0 functions as a bridge. In fact, this is the only strict bridge in this graph.

In [ ]:
is_bridge = [False]*G.vcount()
for v in range(G.vcount()):
    H = G.copy()
    H.delete_vertices(v)
    is_bridge[v] = len(H.clusters()) > 1
    
ig.plot(G, vertex_color=is_bridge, palette=ig.RainbowPalette(2))

# Social network analysis

## Homophily

If two nodes are more likely to be connected when they share a certain attribute, this is known as *homophily*. We will illustrate this on a network of contact between pupils from a high school available from [SocioPatterns](http://www.sociopatterns.org/datasets/high-school-contact-and-friendship-networks/). The contacts were collected through devices that would automatically record face-to-face interaction (i.e. physical proximity) between pupils during 5 days.

In [ ]:
import ighelper
G = ighelper.GraphFromURL('https://storage.googleapis.com/css-files/sociopatterns.graphml')
G = G.clusters().giant()

G.es['width'] = 0.01*np.array(G.es['weight'])
G['layout'] = G.layout_fruchterman_reingold(weights='weight')
G.vs['color'] = ['pink' if v['gender'] == 'F' else 'blue' for v in G.vs]

ig.plot(G)

In the above plot, the male pupils are colored blue, and the females pink. Now the central question is whether males and femals tend to have more links to the same sex, or more evenly distributed. The more general term for homophily is *assortativity*, which refers to how likely two nodes that have the same attribute are linked. In the plot above it is not immediately clear whether there is gender assortativity (or homophily) in this network. But using a statistical measure of assortativity gives us more insight.

In [ ]:
G.assortativity_nominal(ig.VertexClustering.FromAttribute(G, 'gender').membership)

Like a correlation, the assortativity would be 0 if gender would have no relation with the connectivity, 1 if all connections would only be between nodes of the same class and -1 if all connections would only be between nodes of different classes. An assortativity of 0.17 is then not very high, but it does indicate some tendency for pupils of the same sex to be linked.

The pupils mostly associate with other people from the same (school)class. This is again clear from the assortativity based on the (school)class.

In [ ]:
G.assortativity_nominal(ig.VertexClustering.FromAttribute(G, 'class').membership)

If we look at the gender assortativity per class this drops to 0. This suggests that the overall assortativity of 0.17 due to gender imbalance in the classes.

In [ ]:
classes = ig.VertexClustering.FromAttribute(G, 'class');
for H in classes.subgraphs():
    gender_ratio = sum([v['gender'] == 'F' for v in H.vs])/float(H.vcount())
    gender_assortativity = H.assortativity_nominal(ig.VertexClustering.FromAttribute(H, 'gender').membership)
    print('class: {0},\t% female: {1:.2f},\tassortativity: {2: .3f}'.format(H.vs['class'][0], gender_ratio, gender_assortativity))

## Social influence

One of the key concepts in social network analysis is social influence. The basic idea is that people that are linked influence each other. For example, if person A holds opinion A but all his neighbors hold opinion B, then he is likely to switch to opinion A. Let us simulate some simple opinion dynamics on the highschool network. We will assume that every node will switch to the majority opinion in its neighborhood, and we will start from some random initial condition.

In [ ]:
G = ighelper.GraphFromURL('https://storage.googleapis.com/css-files/sociopatterns.graphml')
G = G.clusters().giant()

In [ ]:
G.vs['opinion'] = 0
G.vs['new_opinion'] = np.random.randint(2, size=G.vcount())

while G.vs['opinion'] != G.vs['new_opinion']:
    G.vs['opinion'] = G.vs['new_opinion']
    for v in G.vs:
        v['new_opinion'] = 1*(sum(G.vs[G.neighbors(v)]['opinion']) > 0.5*v.degree())

#G.vs[v.neighbors()]['opinion']
ig.plot(G, vertex_color=G.vs['opinion'], palette=ig.RainbowPalette(2))

One of the problems with social influence is that it usually is confounded with homophily. Let us check if there is assortativity on the result of the opinion dynamics.

In [ ]:
G.assortativity_nominal(G.vs['opinion'])

The assortativity on the opinion is roughly the same as the assortativity on the classes. But then begs the question of whether homophily was operating or social influence. The cases here are clear. The class likely influences the probability of a link, so homophily is operating. In the opinion dynamics simulation, we know that the network did not change, but only the opinion did, so there is clear social influence. But if we empirically see that there is some assortativity on some variable, we cannot determine unambiguously whether there is homophily or social influence.

## Centrality

Sometimes, rather than viewing social influence as a process, it is seen as the overall influence some person can exert over others. It is thought that more central persons are generally more influential. In order to sway opinion, you could for example better target the more central person. However, there are several variants of centrality, and all of them have some merit. We will illustrate them on the karate club network again.

In [ ]:
G = ig.Graph.Famous('Zachary')
G.vs['color'] = 'gray'
G.es['color'] = 'gray'
G['layout'] = G.layout_auto()

### Degree centrality

The simples is degree centrality, which is nothing more than simply the number of contacts somebody has.

In [ ]:
G.vs['centrality'] = G.degree()
ig.plot(G, vertex_size=G.vs['centrality'])

### Eigenvector centrality

The idea of eigenvector centrality is that you are as central as the people you are connected with. So, if you connect to many central people, you should be central yourself. Perhaps this recursive notion of centrality is confusion, but it leads to an elegant solution. If you write out the math, it turns out the largest so-called eigenvector is actually the solution, whence the name.

In [ ]:
G.vs['centrality'] = G.eigenvector_centrality()
ig.plot(G, vertex_size=10*np.array(G.vs['centrality']))

### Pagerank

One problem with eigenvector centrality is that it is not always well-defined in the case of directed graphs. The interesting thing of eigenvector centrality is that it is similar to the proportion of time spent in a node if a *random walker* would traverse the network, choosing links to follow at random. In directed graphs there may be so-called *sinks*: nodes with no outgoing edges. If you think of a random walker, you can imagine the problem: (s)he gets stuck in a sink. The opposite of *sinks* are called *sources*: nodes with no incoming edge. A random walker would then never arrive in a source.

To alleviate this problem, the idea of *teleportation* was introduced. With a small probability a random walker would start in any other node at random. Hence, from any sink node the walker can then always escape, and there is always a probability (s)he arrives at a source node. This was introduced by the founders of Google to model a *random surfer*, jumping from webpage to webpage and every now and then starting anew. This centrality measure is called pagerank, and it forms the core of Google's search engine.

In [ ]:
G.vs['centrality'] = G.pagerank()
ig.plot(G, vertex_size=100*np.array(G.vs['centrality']))

### Betweenness centrality

The previous two centralities were based on random walks in a certain sense. Betweenness centrality in contrast uses the shortest path rather than random walks. The fraction of the shortest paths that goes through a certain node is called the betweenness centrality.

In [ ]:
G.vs['centrality'] = G.betweenness(G.vs)
ig.plot(G, vertex_size=0.1*np.array(G.vs['centrality']) + 1)

As you can see, most centralities tend to give relatively similar results, but there can be some variation.

## Weak ties

There are two main ideas associated with the so-called strenght of weak ties (i.e. links of low weight). The first is that weak ties hold together the network. The second is that new information is obtained through weak ties. The first can be studied relatively easily on any given weighted network. The second is more difficult, and is less frequently studied, because it also requires observations on information sharing.

The idea that weak ties hold together the network can be studied in two contexts. One straightforward possibility is that if we cut weak ties we relatively quickly disconnect the network. Another possibility is that weak ties mostly fall between groups of people. Finally, it is often also analysed whether weak ties mostly fall between nodes that have relatively few common neighbors.

We will study this using the highschool network.

In [ ]:
G = ighelper.GraphFromURL('https://storage.googleapis.com/css-files/sociopatterns.graphml')
G = G.clusters().giant()
G['layout'] = G.layout_fruchterman_reingold(weights='weight')

Let us delete edges from the graph in two different orders: the weakest edges first or the strongest edges first. We look at whether there is a clear giant component, or whether the network splits into more equally sized components.

In [ ]:
H = G.copy()
weak_component_size = [];
while H.ecount() > 0:
    H.delete_edges(min(H.es, key=lambda e: e['weight']))
    #weak_component_size.append(H.clusters().giant().vcount())
    cl = H.clusters()
    weak_component_size.append(sum(np.array(cl.sizes())**2) - cl.giant().vcount()**2)
    
H = G.copy()
strong_component_size = [];
while H.ecount() > 0:
    H.delete_edges(max(H.es, key=lambda e: e['weight']))
    #strong_component_size.append(H.clusters().giant().vcount())    
    cl = H.clusters()
    strong_component_size.append(sum(np.array(cl.sizes())**2) - cl.giant().vcount()**2)

Plotting the results shows that the deleting the weak edges first tends to have a quite different effect than deleting the strong edges first.

In [ ]:
import seaborn as sns
%matplotlib inline
plt.plot(weak_component_size, label='Weak')
plt.plot(strong_component_size, label='Strong')
plt.xlabel('Edge number')
plt.ylabel('LCC size')
plt.legend(loc='best');

The plot may be a bit obscure. Inspecting the actual resulting networks after deleting a critical amount of edges may be more revealing.

In [ ]:
break_point = np.array(weak_component_size).argmax()

Let us first look at the network if we remove the strong links first. It is clear that this network has a single giant component, which is only connected through weak links, and many isolated nodes.

In [ ]:
H = G.copy()
del H['layout']
H.delete_edges(sorted(H.es, key=lambda e: e['weight'], reverse=True)[:break_point])
ig.plot(H.clusters(), edge_width=0.001*np.array(H.es['weight']))

This contrasts sharply to the network if we remove the weak links first. We can clearly see that it seperates into several smaller sizeable components.

In [ ]:
H = G.copy()
del H['layout']
H.delete_edges(sorted(H.es, key=lambda e: e['weight'])[:break_point])
ig.plot(H.clusters(), edge_width=0.001*np.array(H.es['weight']))

There are some obvious groups in this networks, namely the classes. The weak links would then be expected to fall between the different classes, while the stronger links are more likely to be found within the classes. This can easily be computed.

In [ ]:
classes = ig.VertexClustering.FromAttribute(G, 'class')
is_crossing_edge = classes.crossing()
weight_between = np.mean([e['weight'] for e in G.es if is_crossing_edge[e.index]])
weight_within = np.mean([e['weight'] for e in G.es if not is_crossing_edge[e.index]])
print('Weight within {0:.2f}, between {1:.2f}.'.format(weight_within, weight_between))

Finally, we can check if strong links tend to occur between people that have a high overlap of common neighbours. This overlap is usually calculated as the Jaccard similarity.

In [ ]:
import pandas as pd
jaccard_similarity = G.similarity_jaccard(pairs=G.get_edgelist());
df = pd.DataFrame({'jaccard': jaccard_similarity, 
                   'weight': G.es['weight']})
df['jaccard_group'] = np.round(df['jaccard'], 1)

sns.boxplot(x='jaccard_group', y='weight', data=df)
plt.yscale('log')

## Structural holes

When nodes have a high clustering coefficient, they are often thought to be well *embedded* in the network. People in such a position in the network tend to have friends that are also friends themselves. In a certain sense, they are in a *cohesive* environment. This is generally thought to increase trust in one another, but also tends to be associated with higher peer pressure and social norm enforcement.

Node that have a low clustering coefficient occupy a rather different position in the network. They tend to connect different parts of the network, and we earlier saw that they tend to act as bridges. This notion of a bridge can be a bit relaxed to a *local bridge*: an edge without common neighbors, or stated differently a jaccard similarity of 0.

Nodes with such a position are said to be in a position of a *structural hole*. They are the go-between for different people, providing such strutural holes with a strategic advantage. For example, a real-estate agent may facilitate a transaction between a buyer and a seller. Neither buyer nor seller would be connected if it wasn't for the real-estate agent. Indeed, people in such a position of a *structural hole* are sometimes also called *brokers*.

Structural holes are also thought to have more opportunities for innovation and creativity. At the interface of different groups, people occupying structural holes may hear much more, and diverse, information. This could enable them to integrate this diverse information into something new.

Let us identify the brokers in the highschool network.

In [ ]:
G.vs['clustering'] = G.transitivity_local_undirected()
most_broker = min(G.vs, key=lambda v: v['clustering'])

G.vs['color'] = 'gray'
most_broker['color'] = 'red'
G.vs[G.neighbors(most_broker)]['color'] = 'blue'

ig.plot(G, edge_width=0.001*np.array(G.es['weight']), vertex_size=5.0/np.array(G.vs['clustering']))

As can be seen, the node with the lowest clustering tends to connect people from across the network. This contrasts to a node with a higher clustering (but at least as many neighbors).

In [ ]:
least_broker = max(G.vs.select(_degree_ge=most_broker.degree()), key=lambda v: v['clustering'])
                   
G.vs['color'] = 'gray'
least_broker['color'] = 'red'
G.vs[G.neighbors(least_broker)]['color'] = 'blue'

ig.plot(G, edge_width=0.001*np.array(G.es['weight']), vertex_size=5.0/np.array(G.vs['clustering']))

We can make this more explicit. We can calculate the dispersion of neighbors across different classes in the network as the entropy. A higher entropy indicates that the neighbors are more equally distributed across the network, whereas a lower entropy indicates that the neighbors are mostly concentrated in a few classes.

In [ ]:
from collections import Counter

def entropy(lst):
    freq = Counter(lst);
    prob = [float(f)/len(lst) for f in freq.itervalues()]
    return -sum(prob*np.log(prob))

entropy_node = [entropy(G.vs[G.neighbors(v)]['class']) for v in G.vs];

plt.plot(G.vs['clustering'], entropy_node, '.')
plt.xlabel('Clustering')
plt.ylabel('Entropy')

## Equivalence

People occupy different positions in a network. We already saw brokers for example. This idea can be somewhat generalized to the notion of equivalence. The most strict definition of equivalence between two nodes is if they share the exacte same neighbors, called *structural equivalence*. A slightly more relaxed definition is that of *isomorphic equivalence*, meaning that switching the labelling of two nodes that are isomorphically equivalent does not change the structure of the graph. This is still quite restrictive actually, and very hard to compute. 

The most relaxed definition is *regular equivalence*, which considers two nodes as equivalent if they connect to similar others. For example, employees report to their managers who report to the board. Then all managers occupy a regularly equivalent position: they connect to employees and board members. Similarly all employees are regularly equivalent: they all report to managers (even though they don't necessarily report to the same manager). This is a recursive definition, and also is relatively difficult to compute. 

Of course exact regular equivalence may not be the most informative, so some trade-off between the number of 'errors' (i.e. deviating from the other members of the equivalence class) may be tolerated. This is sometimes also referred to as *role detection* as a generalization of *community detection* which we will encounter later on.

Unfortunately there are currently no implementations available for `igraph`, and in fact, no implementations for Python that I know of.

## Social balance

Social balance does not refer to some zen concept of balance or a state of peace. If anything, it actually has more to do with conflict than with peace. Social balance is a theory regarding the structure of *negative links*, i.e. links that have a negative connotation, such as hate or war. These can either be presented as a separate type of link, or as links having some continuous weight, which may also be negative.

# Complex networks

The previous chapter focussed on concepts that originate more in the sociological tradition of network analysis. This chapter will cover aspects that originate in the so-called `complex networks` perspective. These are more recent development, and often have a motivation stemming from the exact sciences. Its outlook is generally more mathematical and statistical, rather than substantive. One challenge is to bridge these two somewhat separate worlds.

## Random graphs

There are various motiviation for studying random graphs. The first is that it provide

## Spreading & percolation

## Community detection